In [25]:
import numpy as np
import networkx as nx

from graphqec import RepetitionCode, RotatedSurfaceCode

The Tanner graph is the representation used here for CSS codes. Identify logical operators using the Tanner graph data structure.

The logic_check is a list of strings, with same structure as the checks.

In [3]:
rep = RepetitionCode(
    distance = 5,
    depolarize1_rate = 0.01,
    depolarize2_rate = 0.01
)

In [26]:
rot = RotatedSurfaceCode(
    distance = 5,
    depolarize1_rate = 0.01,
    depolarize2_rate = 0.01
)

0
1
2
3
4
5
6
7
8


In [17]:
rep.get_check_label

<bound method BaseCode.get_check_label of <graphqec.codes.repetition_code.RepetitionCode object at 0x000001FED1909D50>>

In [23]:
for node in graph.nodes():
    print(nx.get_node_attributes(graph,node))

{}
{}
{}
{}
{}
{}
{}
{}
{}
